## Stable Diffusion Web UI Forge

**Homepage :** [SD Web UI Complete setup](https://github.com/ffxvs/sd-webui-complete-setup)  
**Guide :** [Paperspace Guide](https://github.com/ffxvs/sd-webui-complete-setup/wiki/Paperspace-Guide)

### Check for Updates

**Version : 2024.06.01**

In [ ]:
import requests
import ipywidgets as widgets
from IPython.utils import capture

currentVersion = '2024.06.01'
updateURL = 'https://raw.githubusercontent.com/ffxvs/sd-webui-complete-setup/main/updates.json'
res = requests.get(updateURL)
output = widgets.Output()

def onClick(b, url, version):
    with output:
        print('Downloading...')
        !wget -nv -O /notebooks/sd_webui_forge_paperspace_{version}.ipynb {url}
        print(f'sd_webui_forge_paperspace_{version}.ipynb downloaded in the root directory')

if res.status_code == 200:
    notebook = next((n for n in res.json()['paperspace'] if n['id'] == 'forge'), None)
    if notebook:
        print(f'Current version : {currentVersion}\nLatest version  : {notebook["version"]}')
        if notebook['version'] > currentVersion:
            print('\nThere is new version')
            button = widgets.Button(description="Download now", button_style='success')
            button.on_click(lambda b: onClick(b, notebook['url'], notebook['version']))
            display(button, output)
            print(f'\nChangelog :\n{notebook["changelog"]}')
        else:
            print('\nThis is the latest version')
else:
    print(f'Failed to check for updates\nResponse code : {res.status_code}')

## 1. Requirements

### 1.1. Variables and Functions
**REQUIRED EVERY TIME YOU RUN THIS NOTEBOOK**

In [ ]:
##################################### IMPORT #######################################

import os, re, requests, json
import ipywidgets as widgets
from IPython.utils import capture
from IPython.display import clear_output

############################ ENVIRONMENT VARIABLES ################################

os.environ['LD_PRELOAD'] = '/lib/x86_64-linux-gnu/libtcmalloc.so.4'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['PYTHONWARNINGS'] = 'ignore'
os.environ['PIP_ROOT_USER_ACTION'] = 'ignore'
os.environ['PIP_DISABLE_PIP_VERSION_CHECK'] = '1'
os.environ['FORCE_CUDA'] = '1'
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'garbage_collection_threshold:0.9,max_split_size_mb:512'
os.environ['CUDA_LAUNCH_BLOCKING'] = '0'
os.environ['CUDA_CACHE_DISABLE'] = '0'
os.environ['CUDA_AUTO_BOOST'] = '1'
os.environ['CUDA_MODULE_LOADING'] = 'LAZY'
os.environ['CUDA_DEVICE_DEFAULT_PERSISTING_L2_CACHE_PERCENTAGE_LIMIT'] = '0'
os.environ['GRADIO_ANALYTICS_ENABLED'] = 'False'
os.environ['SAFETENSORS_FAST_GPU'] = '1'
os.environ['NUMEXPR_MAX_THREADS'] = '16'

########################### GLOBAL PATHS AND FUNCTION ###########################

# Paths
root = '/notebooks'
webui = root + '/stable-diffusion-webui-forge'
modulesPath = webui + '/modules'
outputsPath = webui + '/outputs'
extensionsPath = webui + '/extensions'
controlNetModelsPath = webui + "/models/ControlNet"
embeddingsPath = webui + "/embeddings"
modelsPath = webui + "/models/Stable-diffusion"
loraPath = webui + "/models/Lora"
upscalerPath = webui + "/models/ESRGAN"
vaePath = webui + "/models/VAE"
preprocessorPath = modelsPath + '/ControlNetPreprocessor'
sharedStorage = root + '/shared-storage'
sharedModelsPath = sharedStorage + '/models'
sharedEmbeddingsPath = sharedStorage + '/embeddings'
sharedLoraPath = sharedStorage + '/lora'
sharedUpscalerPath = sharedStorage + '/esrgan'
sharedVaePath = sharedStorage + '/vae'
sharedControlNetModelsPath = sharedStorage + '/controlNet'
sharedOutputsPath = sharedStorage + '/outputs'
sharedConfigPath = sharedStorage + '/config'
tempStorage = '/temp-storage'
tempModelsPath = tempStorage + '/models'
tempControlNetModelsPath = tempStorage + '/controlNet'
tempPreprocessorPath = tempControlNetModelsPath + '/preprocessor'

# Resource URLs
mainRepoURL = 'https://raw.githubusercontent.com/ffxvs/sd-webui-complete-setup/main'
defaultExtensionsURL = mainRepoURL + '/res/builtin-extensions.json'
extensionsURL = mainRepoURL + '/res/extensions.json'

boolean = [False, True]
excludeExtensions = ['controlNet']
requestHeaders = {
    "Cache-Control": "no-cache, no-store, must-revalidate",
    "Pragma": "no-cache",
    "Expires": "0"
}

def createTempStorage():
    !mkdir -p {tempModelsPath}/sd {tempModelsPath}/sdxl {tempControlNetModelsPath}/sd {tempControlNetModelsPath}/sdxl {tempPreprocessorPath}

# Create symlink
def symlink(source, destination):
    if os.path.exists(source) and not os.path.islink(destination):
        !rm -r -f {destination}
        !ln -s {source} {destination}

# Remove symlink
def unlink(target):
    if os.path.exists(target) and os.path.islink(target):
        !unlink {target}

# Complete message
def completedMessage(): 
    completed = widgets.Button(description='Completed', button_style='success', icon='check')
    print('\n')
    display(completed)

# Sync Configs
def syncConfigs():
    configFile = f'{sharedConfigPath}/config.json'
    if os.path.exists(configFile):
        with open(configFile, 'r') as file:
            config = json.load(file)
        config['samples_filename_pattern'] = "[datetime<%d-%m-%Y_%H%M%S>]"
        config['cross_attention_optimization'] = "xformers"
        config['controlnet_clip_detector_on_cpu'] = True
        removeKeys = ["bmab_cn_openpose", "bmab_cn_lineart", "bmab_cn_inpaint", "bmab_cn_tile_resample"]
        for key in removeKeys:
            config.pop(key, None)
        with open(configFile, 'w') as file:
            json.dump(config, file, indent=4)

# Hide samplers
def hideSamplers():
    configFile = f'{sharedConfigPath}/config.json'
    if os.path.exists(configFile):
        with open(configFile, 'r') as file:
            config = json.load(file)
        samplersToHide = ["DPM fast", "DPM++ SDE", "DPM++ 2M", "LMS", "DPM2", "DPM++ 2M SDE", "PLMS", "DPM++ 2M SDE Heun"]
        for sampler in samplersToHide:
            if sampler not in config['hide_samplers']:
                config['hide_samplers'].append(sampler)
        with open(configFile, 'w') as file:
            json.dump(config, file, indent=4)

# Shared & temporary storage symlinks 
def tempStorageSymlink(var, source, destination):
    if var:
        symlink(source, destination)
    else:
        unlink(destination)
        !rm -r -f {source}/* && mkdir -p {destination}

def storageSymlinks():
    # Make symlinks from temporary storage
    tempStorageSymlink(temp_sd15_models, f'{tempModelsPath}/sd', f'{sharedModelsPath}/sd')
    tempStorageSymlink(temp_sdxl_models, f'{tempModelsPath}/sdxl', f'{sharedModelsPath}/sdxl')
    tempStorageSymlink(temp_controlnet_sd15, f'{tempControlNetModelsPath}/sd', f'{sharedControlNetModelsPath}/sd')
    tempStorageSymlink(temp_controlnet_sdxl, f'{tempControlNetModelsPath}/sdxl', f'{sharedControlNetModelsPath}/sdxl')
    tempStorageSymlink(temp_preprocessor, tempPreprocessorPath, preprocessorPath)

    # Make symlinks from shared storage
    symlink(sharedModelsPath, modelsPath)
    symlink(sharedLoraPath, loraPath)
    symlink(sharedEmbeddingsPath, embeddingsPath)
    symlink(sharedUpscalerPath, upscalerPath)
    symlink(sharedVaePath, vaePath)
    symlink(sharedControlNetModelsPath, controlNetModelsPath)
    symlink(sharedOutputsPath, outputsPath)
    symlink(f'{sharedConfigPath}/config.json', f'{webui}/config.json')
    symlink(f'{sharedConfigPath}/ui-config.json', f'{webui}/ui-config.json')

# Resource
def getResource(url):
    res = requests.get(url, headers=requestHeaders)
    if res.status_code == 200:
        return res.json()
    else:
        return False

# Aria2c
def downloader(url, path, overwrite=False):
    args = '--download-result=hide --console-log-level=error -c -x 16 -s 16 -k 1M '
    if overwrite: args += '--allow-overwrite'
    formattedURL = '"' + url + '"'
    if bool(re.search(r'\/[\w\.-]+\.\w+$', url)):
        filename = url.split('/')[-1]
        !aria2c {args} {formattedURL} -d {path} -o {filename}
    else:
        !aria2c {args} {formattedURL} -d {path}

# Git Clone
def silentClone(command, path, update=False, overwrite=False):
    directory = command.split('/')[-1]
    if os.path.exists(path + '/' + directory):
        if update:
            os.chdir(f'{path}/{directory}')
            !git pull -q
        elif overwrite:
            !rm -r {path}/{directory}
            !git clone -q --depth 10 {command} {path}/{directory}
    else:
        !git clone -q --depth 10 {command} {path}/{directory}

# WGet
def silentGet(command):
    !wget -nv {command}

def downloadDefaultExtensions():
    print("⏳ Installing built-in extensions...")
    for ext in getResource(defaultExtensionsURL)['extensions']:
        if not ext['id'] in excludeExtensions:
            print(ext['name'] + '...')
            silentClone(ext['url'], extensionsPath, updateExts)

def downloadExtensions():
    print("\n⏳ Installing selected extensions...")
    for ext in getResource(extensionsURL)['extensions']:
        try:
            if eval(ext['id']):
                if ext['id'] == 'bmab':
                    !pip install -q basicsr
                print(ext['name'] + '...')
                silentClone(ext['url'], extensionsPath, updateExts)
        except:
            pass

def downloadOtherExtensions(extensions):
    if extensions:
        print("⏳ Installing extensions...")  
        for ext in extensions:
            name = ext.split('/')[-1]
            print(name + '...')
            silentClone(ext, extensionsPath, updateExts)

def launchWebui():
    print('⏳ Preparing...')
    print('It will take a little longer...')
    args = '--disable-console-progressbars --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-hashing --api --xformers'
    blocksPath = '/usr/local/lib/python3.10/dist-packages/gradio/blocks.py'
    os.chdir(webui)

    if darkTheme:
        args += ' --theme dark'
    if username and password:
        args += f' --gradio-auth {username}:{password}'
    try:
        if ngrokToken:
            args += f' --ngrok {ngrokToken}'
            if ngrokDomain:
                ngrokOptions = '\'{"hostname":"' + ngrokDomain + '"}\''
                args += f' --ngrok-options {ngrokOptions}'
    except:
        pass
    if cors:
        args += f' --cors-allow-origins {cors}'

    with capture.capture_output() as cap:
        !python launch.py {args} --exit
        !pip install -q pillow==9.5.0

    with open(blocksPath, 'r') as file:
        content = file.read()

    pattern = re.compile(r'print\(\s*strings\.en\["RUNNING_LOCALLY_SEPARATED"\]\.format\(\s*self\.protocol, self\.server_name, self\.server_port\s*\)\s*\)')
    replace = re.sub(pattern, 'print(strings.en["RUNNING_LOCALLY"].format(f\'https://tensorboard-{os.environ.get("PAPERSPACE_FQDN")}\'))', content)
    
    with open(blocksPath, 'w') as file:
        file.write(replace)

    args += ' --listen --port 6006'
    print('Launching Web UI...')
    !python webui.py {args}

syncConfigs()
hideSamplers()
createTempStorage()
completedMessage()

### 1.2. Dependencies
**REQUIRED EVERY TIME YOU START THE MACHINE**

In [ ]:
print('⏳ Installing dependencies...')
os.chdir(root)

with capture.capture_output() as cap:
    !apt -y -q update
    !apt -y -q install gcc g++ google-perftools
    !pip install torchaudio==2.1.2 --index-url https://download.pytorch.org/whl/cu121
    !pip install setuptools==69.5.1

completedMessage()

### 1.3. Shared Storage
**Only needs to be run once on the first installation**

In [ ]:
print('⏳ Creating shared storage directory...')
# Make shared storage symlink
os.chdir('/')
symlink('/storage', sharedStorage)

# Make necessary folders in shared storage if not exists
os.chdir(sharedStorage)
!mkdir -p {sharedModelsPath}/sd {sharedModelsPath}/sdxl
!mkdir -p {sharedEmbeddingsPath}/sd {sharedEmbeddingsPath}/sdxl
!mkdir -p {sharedLoraPath}/sd {sharedLoraPath}/sdxl
!mkdir -p {sharedVaePath}/sd {sharedVaePath}/sdxl
!mkdir -p {sharedControlNetModelsPath}/sd {sharedControlNetModelsPath}/sdxl
!mkdir -p {sharedUpscalerPath}
!mkdir -p {sharedOutputsPath}
!mkdir -p {sharedConfigPath}

os.chdir(root)
completedMessage()

## 2. Setup Web UI

### 2.1. Web UI

In [ ]:
# Update Web UI
updateWebui = boolean[0]

# Temporary Storage - store some resources in temporary storage to avoid storage charges
temp_sd15_models = boolean[0]
temp_sdxl_models = boolean[0]
temp_controlnet_sd15 = boolean[0]
temp_controlnet_sdxl = boolean[0]
temp_preprocessor = boolean[0]

################################################################################################################

os.chdir(root)
print('⏳ Installing Stable Diffusion Web UI Forge...')
silentClone('https://github.com/lllyasviel/stable-diffusion-webui-forge', root, updateWebui)
os.chdir(webui)

# Download configs
if not os.path.exists(f'{sharedConfigPath}/config.json'):
    downloader(f'{mainRepoURL}/configs/config.json', sharedConfigPath)
if not os.path.exists(f'{sharedConfigPath}/ui-config.json'):
    downloader(f'{mainRepoURL}/configs/ui-config.json', sharedConfigPath)

storageSymlinks()
completedMessage()

### 2.2. Extensions
Since this is a fork of Stable Diffusion Web UI, some extensions might not work properly.  
If you find any extension error, please report it to me on github.

#### • Built-in Extensions
**This is just for information. No need to run this cell.**

* [Aspect Ratio Helper](https://github.com/thomasasfk/sd-webui-aspect-ratio-helper)
* [Canvas Zoom](https://github.com/richrobber2/canvas-zoom)
* [Cleaner](https://github.com/novitalabs/sd-webui-cleaner)
* [Infinite Image Browsing](https://github.com/zanllp/sd-webui-infinite-image-browsing)
* [SD Delete Button](https://github.com/reforget-id/sd_delete_button)
* [State](https://github.com/ilian6806/stable-diffusion-webui-state)
* [Ultimate SD Upscale](https://github.com/Coyote-A/ultimate-upscale-for-automatic1111)

#### • Extension List

In [ ]:
# UPDATE EXTENSIONS
updateExts = boolean[0]

# Adetailer (After Detailer) - https://github.com/Bing-su/adetailer
adetailer = boolean[1]

# AnimateDiff - https://github.com/continue-revolution/sd-webui-animatediff
animateDiff = boolean[0]

# BMAB - https://github.com/portu-sim/sd-webui-bmab
bmab = boolean[0]

# Depth Map Library - https://github.com/wywywywy/sd-webui-depth-lib
depthLib = boolean[0]

# Hugging Face - https://github.com/camenduru/stable-diffusion-webui-huggingface
huggingFace = boolean[0]

# Inpaint Anything - https://github.com/Uminosachi/sd-webui-inpaint-anything
inpaintAny = boolean[0]

# Latent Couple - https://github.com/aria1th/stable-diffusion-webui-two-shot
latentCouple = boolean[0]

# Mini Paint - https://github.com/0Tick/a1111-mini-paint
miniPaint = boolean[1]

# SD Model Downloader - https://github.com/Iyashinouta/sd-model-downloader
modelDownloader = boolean[0]

# Negative Prompt Weight - https://github.com/muerrilla/stable-diffusion-NPW
npw = boolean[0]

# openOutpaint - https://github.com/zero01101/openOutpaint-webUI-extension
openOutpaint = boolean[0]

# Photopea - https://github.com/yankooliveira/sd-webui-photopea-embed
photopea = boolean[1]

# Prompt History - https://github.com/namkazt/sd-webui-prompt-history
promptHistory = boolean[0]

# Regional Prompter - https://github.com/hako-mikan/sd-webui-regional-prompter
regionalPrompter = boolean[0]

# Remove Background - https://github.com/AUTOMATIC1111/stable-diffusion-webui-rembg
rembg = boolean[0]

# Style Selector XL - https://github.com/ahgsql/StyleSelectorXL
styleXL = boolean[0]

# Tag Autocomplete - https://github.com/DominikDoom/a1111-sd-webui-tagcomplete
tagComplete = boolean[1]


################################################################################################################

downloadDefaultExtensions()
downloadExtensions()
os.chdir(webui)
completedMessage()

#### • Install from URLs

In [ ]:
# UPDATE EXTENSIONS
updateExts = boolean[0]

# Install extensions from URL
otherExtensions = []


################################################################################################################

downloadOtherExtensions(otherExtensions)
os.chdir(webui)
completedMessage()

## 3. Launch Web UI
Before running this cell, download models using `sd15_resource_lists.ipynb` or `sdxl_resource_lists.ipynb`.  
Run this cell and wait until `Model loaded in **.*s ...` appears.  
Then, click link `https://tensorboard-xxxxx.paperspacegradient.com` or ngrok link `https://xxxxx.ngrok-free.app` to open Web UI.  
Always watch the output log to track image generation progress or errors. 

In [ ]:
# Dark theme
darkTheme = boolean[1]

# Authentication (Recommended for security purposes)
username = ''
password = ''

# Ngrok (Optional) 
# Some users reported that using ngrok can get you banned. So, this option will be disabled temporarily. 
# If you want to take the risk or have never had problems with ngrok before, you can remove the hashtags.
# ngrokToken = ''
# ngrokDomain = ''

# CORS (Optional) 
# separate with commas
cors = 'https://huchenlei.github.io'


################################################################################################################

launchWebui()